In [13]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import VGG16
import requests
import zipfile
import io
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [14]:
data_url = 'https://github.com/Griselia23/wastemanagementapk/raw/main/ML/dataset/trash.zip'

# Download and extract data
response = requests.get(data_url)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extractall('trash_dataset')

data_dir = 'trash_dataset/dataset-resized'

In [15]:
def create_data_generators(data_dir, target_size, batch_size):
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        validation_split=0.2
    )
    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    validation_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, validation_generator

In [16]:
# Define target size and batch size
target_size = (224, 224)
batch_size = 32

# Create data generators
train_generator, validation_generator = create_data_generators(data_dir, target_size, batch_size)

Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.


In [17]:
def create_model(input_shape, num_classes):
    # Use pre-trained VGG16 model as base
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze convolutional layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom classification head
    x = Flatten()(base_model.output)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=output)
    
    return model

In [18]:
# Define input shape and number of classes
input_shape = (224, 224, 3)
num_classes = 6

In [19]:
# Create the model
model = create_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [20]:

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
callbacks = [early_stopping, reduce_lr]

In [21]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=callbacks
)

Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 687s 11s/step - accuracy: 0.3007 - loss: 2.2651 - val_accuracy: 0.5042 - val_loss: 1.3198 - learning_rate: 0.0010
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 15s 108ms/step - accuracy: 0.3750 - loss: 1.3341 - val_accuracy: 0.3913 - val_loss: 1.4975 - learning_rate: 0.0010
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 675s 11s/step - accuracy: 0.4307 - loss: 1.3690 - val_accuracy: 0.4979 - val_loss: 1.2870 - learning_rate: 0.0010
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 15s 111ms/step - accuracy: 0.2812 - loss: 1.5719 - val_accuracy: 0.6522 - val_loss: 0.9716 - learning_rate: 0.0010
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 654s 10s/step - accuracy: 0.4327 - loss: 1.3534 - val_accuracy: 0.5271 - val_loss: 1.1501 - learning_rate: 0.0010
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 98ms/step - accuracy: 0.4375 - loss: 1.3167 - val_accuracy: 0.4348 - val_loss: 1.1961 - learning_rate: 0.0010
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 660s 10s/step - accuracy: 0.4489 - loss: 1.2772 

KeyboardInterrupt: 

In [ ]:
# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()